# Mock

In [1]:
import random

class Riddler:

    tries = ['попыток', 'попытка', 'попытки']

    def __init__(self):
        self.riddles = {'Маленький, серенький, на слона похож.': 'слоненок', 
                        'Над нами кверху ногами.': 'таракан', 
                        'Cиний, большой, с усами и полностью набит зайцами.': 'троллейбус'}

    def add_riddle(self, riddle: str, answer: str):
        """ Добавляет загадку в словарь """
        if not isinstance(riddle, str) or not isinstance(answer, str):
            print('Wrong type!!')
            return
        self.riddles[riddle] = answer

    def riddle(self):
        """ Печатает текст загадки и проверяет правильность ответов """
        question = random.choice(list(self.riddles.keys()))
        print('Загадка: ' + question)
        print('У вас 3 попытки!')
        for i in range(3,0, -1):
            answer = input()
            if answer == self.riddles[question]:
                print('Правильно!!!')
                return True
            print(f'У вас {i-1} {self.tries[i-1]}!')
        print('Правильный ответ: ' + self.riddles[question])
        return False
            

In [2]:
riddler = Riddler()

In [3]:
riddler.riddle()

Загадка: Над нами кверху ногами.
У вас 3 попытки!
небо
У вас 2 попытки!
солнце
У вас 1 попытка!
таракан
Правильно!!!


True

In [4]:
import unittest

class RiddlerTestCase(unittest.TestCase):

    def setUp(self):
        self.riddler = Riddler()

    # тестируем метод add_riddle, все довольно просто
    def test_add_riddle_success(self):
        self.riddler.add_riddle('test', 'test')
        self.assertEqual(self.riddler.riddles['test'], 'test')

    def test_add_riddle_wrong_type(self):
        riddles_before = self.riddler.riddles.copy()
        self.riddler.add_riddle(123, 123)
        self.assertEqual(self.riddler.riddles, riddles_before)

    # как тестировать метод с пользовательским вводом? принтом? и рандомом?
    def test_riddle(self):
        pass



if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter


...

Wrong type!!



----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


## unittest.mock.Mock

Mock - специальный объект на любой вызов, обращение к атрибуту или методу возвращающий новый объект Mock или то, что мы сами попросим. 

In [5]:
from unittest.mock import Mock

In [6]:
m = Mock()

In [7]:
# вызов
m()

<Mock name='mock()' id='140608707567056'>

In [10]:
# атрибут
m.attr123456

<Mock name='mock.attr123456' id='140608707129744'>

In [13]:
# метод
m.method134()

<Mock name='mock.method134()' id='140608788788880'>

Можно задавать свои атрибуты и их значения

In [14]:
m = Mock(my_attr=28, my_attr2=128)

In [17]:
m.my_attr3

<Mock name='mock.my_attr3' id='140608707162640'>

### return_value

В параметр return_value передаем то, что хотим получить в результате вызова мок-объекта

In [18]:
m = Mock(return_value=28)

In [19]:
m()

28

### side_effect

В параметр side_effect можно передать много чего: 
 + любой итерируемый объект (тогда мок при каждом вызове будет возвращать следующий элемент итератора)
 + функцию, которая будет вызвана с переданными в исходную функцию парамерами вместо нее
 + или исключение (тогда оно будет поднято в процессе выполнения теста)       
 
Подробнее в документации https://docs.python.org/3/library/unittest.mock.html#unittest.mock.Mock.side_effect


Итерируемый объект:

In [20]:
m = Mock(side_effect=[1,2,3,4,5])

In [21]:
m()

1

In [22]:
m()

2

In [23]:
m()

3

In [25]:
m = Mock(side_effect=[1,2,3,4,5])
for i in range(5):
    print(m())

1
2
3
4
5


Функция:

In [26]:
def side_effect_callable(arg):
    values = {'a': 1, 'b': 2, 'c': 3}
    if arg in values:
        return values[arg]
    return 0

In [27]:
m = Mock(side_effect=side_effect_callable)

In [28]:
m('a')

1

In [29]:
m('c')

3

In [30]:
m('d')

0

In [32]:
m('g')

0

### Проверка списка вызовов

Можно проверять сколько раз и с какими аргументами был бызван мок-объект:

In [33]:
m.call_args_list # список вызовов

[call('a'), call('c'), call('d'), call('g')]

In [34]:
m.assert_called() # был когда-либо вызван

In [35]:
m.assert_called_once() # вызван ровно 1 раз

AssertionError: Expected 'mock' to have been called once. Called 4 times.

In [37]:
m.assert_called_with('g') # проверяет аргументы последнего вызова

In [41]:
from unittest.mock import call
# проверить, что в списке вызовов есть все нужные вызовы в заданном порядке
m.assert_has_calls([call('a'), call('c'), call('d'), call('g')]) 

## подмена объектов с помощью patch

+ заменить один объект дургим на время тестов можно с помощью функции ***patch***
+ используется внутри менеджера контекстов
```
with patch('module.object.method', ...):
    ...
```
+ или в виде декоратора - аргументы будут те же
```
@patch('module.object.method', ...)
def test_something(...):
    ...
```
+ первый аргумент - путь до объекта/метода который надо заменить (через точки, так же как мы импортируем объекты)
+ следующие аргументы определяют на что и как именно заменить
+ подробнее про аргументы: https://docs.python.org/3/library/unittest.mock.html#patch   


**ВАЖНО - где именно заменять объект**
+ То есть какой именно путь писать в ***patch*** первым аргументом?
+ Основное правило - заменять объект нужно **там где он используется**, а не там откуда его импортировали. 
+ То есть если в модуле (***my_beautiful_module.py***), который мы хотим протестировать импортируется какой-то объект который мы хотим заменить на мок (в данном случае функция ***some_fucntion***)
```
from some_module import some_function
def my_function():
    result = some_method() + 1
    return result
```
+ То в тестах нужно делать вот так, (а не *'some_module.some_function'*)
```
@patch('my_beautiful_module.some_function', ...)
def test_my_function(...):
    ...
```
+ Обычно все работает, даже если делать неправильно, но далеко не всегда.
+ Подробнее про это: https://docs.python.org/3/library/unittest.mock.html#id6


In [1]:
def greet_user():
    name = input('Представьтесь, пожалуйста')
    return('Привет, %s!' % name)

In [3]:
import unittest
from unittest.mock import Mock, patch

class GreetUserTestCase(unittest.TestCase):
    
    @patch('__main__.input', Mock(return_value='Юрий'))
#     def test_greet_user(self):
#         self.assertEqual(greet_user(), 'Привет, Юрий!')
    
    # эквивалентно
    def test_greet_user(self):
        with patch('builtins.input', Mock(return_value='Юрий')) as mock_input:
            self.assertEqual(greet_user(), 'Привет, Юрий!')
        
    
if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


**Задание**: 
+ переписать тесты для метода greet_user, при условии замены ***return*** на ***print***
+ обязательно проверить, что именно выводится на экран!

In [2]:
def greet_user():
    name = input('Представьтесь, пожалуйста')
    print('Привет, %s!' % name)

In [7]:
import unittest
from unittest.mock import Mock, patch


class GreetUserTestCase(unittest.TestCase):
    
    # способ 1
    def setUp(self):
        self.print_mock = Mock()
        
    # 1.1
    @patch('__main__.input', Mock(return_value='Юрий'))
    def test_greet_user(self):
        with patch('__main__.print', self.print_mock):
            greet_user()
            self.print_mock.assert_called_once_with('Привет, Юрий!')
        
#     # 1.2
#     def test_greet_user(self):
#         with patch('__main__.input', Mock(return_value='Юрий')):
#             with patch('__main__.print', self.print_mock):
#                 greet_user()
#                 self.print_mock.assert_called_once_with('Привет, Юрий!')
    
    
#     # способ 2
#     def test_greet_user(self):
#         with patch('__main__.print', Mock()) as mock_print:
#             with patch('__main__.input', Mock(return_value='Юрий')):
#                 greet_user()
#                 mock_print.assert_called_once_with('Привет, Юрий!')
                
#     # способ 3
#     @patch('__main__.input', Mock(return_value='Юрий'))
#     def test_greet_user(self):
#         with patch('__main__.print', Mock()) as mock_print:
#             greet_user()
#             mock_print.assert_called_once_with('Привет, Юрий!')
            
    
        
if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter    

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


**Задание**:
+ переварить все рассказанное и показанное выше и написать тесты для метода ***riddle***
+ проверить нужно все варианты (угадывание с n-ой попытки, неугадывание) и не только возвращаемое значение, но и побочные эффекты (что печатается)
+ добиться 100% покрытия кода тестами

In [16]:
import unittest
from parameterized import parameterized
from unittest.mock import Mock, patch, call

question = list(Riddler().riddles.keys())[0] # зафиксировали вопрос
answer = Riddler().riddles[question] # зафиксировали ответ на него
possible_prints = ['Загадка: ' + question, 
                   'У вас 3 попытки!',
                   'У вас 2 попытки!',
                   'У вас 1 попытка!',
                   'У вас 0 попыток!']
wrong = ['Правильный ответ: ' + answer]
right = ['Правильно!!!']
possible_inputs = ['неправильный ответ 1', 
                   'неправильный ответ 2',
                   'неправильный ответ 3',
                    answer]

def mock_random(iterable):
    return question

class RiddlerTestCase(unittest.TestCase):
    def setUp(self):
        self.riddler = Riddler()

    # тестируем метод add_riddle, все довольно просто
    def test_add_riddle_success(self):
        self.riddler.add_riddle('test', 'test')
        self.assertEqual(self.riddler.riddles['test'], 'test')

    # сюда добавим еще проверку вывода
    def test_add_riddle_wrong_type(self):
        riddles_before = self.riddler.riddles.copy()
        with patch('builtins.print', Mock()) as mock_print:
            self.riddler.add_riddle(123, 123)
            self.assertEqual(self.riddler.riddles, riddles_before)
            # проверка вывода
            mock_print.assert_called_once_with('Wrong type!!')
            
    @parameterized.expand([
        # угадали с 1 раза
        (possible_inputs[-1:], possible_prints[:2]+right, True), 
        # угадали со 2 раза
        (possible_inputs[-2:], possible_prints[:3]+right, True), 
        # угадали с 3 раза
        (possible_inputs[-3:], possible_prints[:4]+right, True), 
        # не угадали 
        (possible_inputs[:-1], possible_prints+wrong, False)])
    @patch('random.choice', Mock(side_effect=mock_random))
    def test_riddle_success(self, expected_input, expected_prints, expected_result):
        with patch('builtins.input', Mock(side_effect=expected_input)):
            with patch('builtins.print', Mock()) as print_mock:
                result = self.riddler.riddle()
            self.assertEqual(result, expected_result)
            print_mock.assert_has_calls([call(arg) for arg in 
                                         expected_prints]) 
    
if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter    

......
----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK


+ Порядок применения декораторов - от внутреннего (ближнего к определению функции) к дальнему
+ Обратите внимание, что декоратор parameterized должен применяться последним, чтобы все работало
+ Подробнее про это: https://medium.com/uckey/python-parameterized-module-is-useful-but-some-limitations-are-there-6afb034c8034